# Baseline Model

In [1]:
import keras
import random
import tensorflow as tf
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from keras.utils.generic_utils import get_custom_objects
from keras.optimizers import Adam, Nadam
from keras.callbacks import Callback
from sklearn.metrics import roc_auc_score

2023-04-06 13:16:43.024523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
import numpy as np

In [15]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
np.random.seed(42) # NumPy
random.seed(42) # Python
tf.random.set_seed(42) # Tensorflow

## Importing Data

In [7]:
train_data = pd.read_pickle('train_knn_scaled.pkl')
test_data = pd.read_pickle('test_knn_scaled.pkl')

In [8]:
train_data.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,card1_div_1000,card2_div_10,R_parent_domain,R_domain_name,R_top_level_domain,device_name,device_version,os_name,os_version,isFraud
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,-1.577987,-0.278167,4,3417,500,42,1,38,1,166,...,4,51,16,24,8,13,293,7,68,0
2987001,-1.577986,-0.443327,4,7922,303,42,2,2,1,173,...,10,30,16,24,8,13,293,7,68,0
2987002,-1.577972,-0.317889,4,9383,389,42,4,58,2,178,...,12,39,16,24,8,13,293,7,68,0
2987003,-1.577965,-0.355521,4,6991,466,42,2,14,2,282,...,9,46,16,24,8,13,293,7,68,0
2987004,-1.577964,-0.355521,1,9262,413,42,2,2,1,241,...,12,41,16,24,8,7,249,0,58,0


In [9]:
test_data.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,P_parent_domain,P_domain_name,P_top_level_domain,R_parent_domain,R_domain_name,R_top_level_domain,device_name,device_version,os_name,os_version
TransactionID,,,,,,,,,,,,,,,,,,,,,
3663549,2.389081,-0.430993,4,365,10,42,4,108,2,52,...,12,15,2,16,24,8,13,293,7,68
3663550,2.389090,-0.359702,4,9099,10,42,4,108,2,151,...,2,2,2,16,24,8,13,293,7,68
3663551,2.389100,0.150412,4,9245,473,42,4,108,2,280,...,15,17,2,16,24,8,13,293,7,68
3663552,2.389100,0.626866,4,870,259,42,4,58,2,79,...,12,15,2,16,24,8,13,293,7,68
3663553,2.389101,-0.280467,4,6894,351,42,2,14,2,127,...,12,15,2,16,24,8,13,293,7,68


In [10]:
X = train_data.drop(['isFraud'], axis=1)
y = train_data['isFraud']

In [11]:
X_test = test_data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

In [20]:
# define the ANN model
model = keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=[X_train.shape[1]]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Recall(),
                       tf.keras.metrics.AUC()])

# fit the model on the training data
model.fit(X_train, y_train, epochs=50, batch_size=2048, validation_data=(X_val, y_val), verbose=1)

Epoch 1/50
923/923 [==============================] - 142s 154ms/step - loss: 4.7195 - recall_2: 0.1351 - false_positives_2: 14182.0000 - auc_2: 0.5650 - val_loss: 0.3274 - val_recall_2: 0.1870 - val_false_positives_2: 706.0000 - val_auc_2: 0.6427
Epoch 2/50
923/923 [==============================] - 6s 6ms/step - loss: 0.8712 - recall_2: 0.1932 - false_positives_2: 10129.0000 - auc_2: 0.6312 - val_loss: 0.4561 - val_recall_2: 0.2204 - val_false_positives_2: 3672.0000 - val_auc_2: 0.6165
Epoch 3/50
923/923 [==============================] - 7s 8ms/step - loss: 0.6330 - recall_2: 0.1982 - false_positives_2: 7955.0000 - auc_2: 0.6547 - val_loss: 0.1372 - val_recall_2: 0.2715 - val_false_positives_2: 955.0000 - val_auc_2: 0.7983
Epoch 4/50
923/923 [==============================] - 6s 7ms/step - loss: 0.3484 - recall_2: 0.1941 - false_positives_2: 4974.0000 - auc_2: 0.6916 - val_loss: 0.1263 - val_recall_2: 0.1798 - val_false_positives_2: 211.0000 - val_auc_2: 0.7796
Epoch 5/50
923/923 [=

In [23]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

In [21]:
from sklearn.metrics import roc_auc_score, recall_score, confusion_matrix

class EvaluationMetrics(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs={}):
        x_val, y_val = self.validation_data
        y_pred = self.model.predict(x_val)

        roc_auc = roc_auc_score(y_val, y_pred)
        logs['val_roc_auc'] = roc_auc

        y_pred_labels = (y_pred > 0.5).astype(int)
        recall = recall_score(y_val, y_pred_labels)
        logs['val_recall'] = recall

        tn, fp, fn, tp = confusion_matrix(y_val, y_pred_labels).ravel()
        false_positive_rate = fp / (fp + tn)
        logs['false_positive_rate'] = false_positive_rate

        print(f'ROC-AUC: {roc_auc:.4f} - Recall: {recall:.4f} - False Positive Rate: {false_positive_rate:.4f}')


In [24]:
checkpoint_callback = ModelCheckpoint(
    filepath="model_checkpoint.h5", # where to save the model checkpoint
    monitor="val_recall", # monitor the validation recall
    save_best_only=True, # save only the best performing model
    save_weights_only=True, # save only the weights of the model, not the entire model
    mode="max", # maximize the validation recall
    verbose=1 # print messages when saving the model
)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [25]:
model2 = keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=[X_train.shape[1]]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model2.compile(optimizer='adam', loss='binary_crossentropy')

In [26]:
evaluation_metrics_callback = EvaluationMetrics(validation_data = (X_val, y_val))

In [27]:
model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=1024, verbose=1, callbacks=[evaluation_metrics_callback, checkpoint_callback])

Epoch 1/50
3691/3691 [==============================] - 4s 917us/step
ROC-AUC: 0.7517 - Recall: 0.0970 - False Positive Rate: 0.0011

Epoch 1: val_recall improved from -inf to 0.09702, saving model to model_checkpoint.h5
462/462 [==============================] - 292s 632ms/step - loss: 0.2361 - val_loss: 0.1425 - val_roc_auc: 0.7517 - val_recall: 0.0970 - false_positive_rate: 0.0011
Epoch 2/50
3691/3691 [==============================] - 3s 900us/step
ROC-AUC: 0.8178 - Recall: 0.1091 - False Positive Rate: 0.0006

Epoch 2: val_recall improved from 0.09702 to 0.10912, saving model to model_checkpoint.h5
462/462 [==============================] - 156s 338ms/step - loss: 0.1347 - val_loss: 0.1197 - val_roc_auc: 0.8178 - val_recall: 0.1091 - false_positive_rate: 5.6153e-04
Epoch 3/50
3691/3691 [==============================] - 3s 871us/step
ROC-AUC: 0.8278 - Recall: 0.1638 - False Positive Rate: 0.0014

Epoch 3: val_recall improved from 0.10912 to 0.16380, saving model to model_checkpoin

KeyboardInterrupt: 